# previous

La table previous_application regroupe toutes les demandes de crédit antérieures du client. Nous avons nettoyé les montants, les statuts, les durées et les types de contrats, puis agrégé ces informations au niveau SK_ID_CURR. Les agrégations incluent les montants demandés et accordés, les écarts entre montants, les durées moyennes, les taux d’approbation et les fréquences des différents types de contrats. Des flags indiquent l’absence d’historique de demandes précédentes. Cette table fournit un résumé clair du comportement passé du client lors de ses précédentes demandes de crédit.


In [3]:

from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")


CWD = Path.cwd()
PROJECT_ROOT = CWD.parent.parent

DATA_RAW       = PROJECT_ROOT / "data" / "raw"
DATA_CLEAN     = PROJECT_ROOT / "data" / "clean"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

DATA_CLEAN.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

print("DATA_RAW       =", DATA_RAW)
print("DATA_PROCESSED =", DATA_PROCESSED)


POS_PATH   = DATA_PROCESSED / "pos_cash_by_prev.csv"
INST_PATH  = DATA_PROCESSED / "installments_payments_final.csv"
CC_PATH    = DATA_PROCESSED / "credit_card_balance_final.csv"
PREV_PATH  = DATA_RAW / "previous_application.csv"

pos = pd.read_csv(POS_PATH)
inst = pd.read_csv(INST_PATH)
cc = pd.read_csv(CC_PATH)
prev_app = pd.read_csv(PREV_PATH)
print("pos :", pos.shape)
print("inst:", inst.shape)
print("cc  :", cc.shape)
print("prev:", prev_app.shape)
prev_clean = prev_app.copy()

DATA_RAW       = c:\Users\yoann\Documents\open classrooms\projet 6\pret a depenser v2\data\raw
DATA_PROCESSED = c:\Users\yoann\Documents\open classrooms\projet 6\pret a depenser v2\data\processed
pos : (936325, 32)
inst: (997752, 50)
cc  : (104307, 129)
prev: (1670214, 37)


## Typage

In [4]:
prev_clean["SK_ID_PREV"] = pd.to_numeric(prev_clean["SK_ID_PREV"], errors="coerce").astype("Int64")
prev_clean["SK_ID_CURR"] = pd.to_numeric(prev_clean["SK_ID_CURR"], errors="coerce").astype("Int64")

# Colonnes "int-like" (on reste en Int64 pour tolérer NA)
int_like = ["HOUR_APPR_PROCESS_START", "DAYS_DECISION", "SELLERPLACE_AREA", "NFLAG_LAST_APPL_IN_DAY"]
for col in int_like:
    if col in prev_clean.columns:
        prev_clean[col] = pd.to_numeric(prev_clean[col], errors="coerce").astype("Int64")

In [5]:
# Colonnes numériques à convertir proprement
num_cols = [
    "AMT_ANNUITY", "AMT_APPLICATION", "AMT_CREDIT",
    "AMT_DOWN_PAYMENT", "AMT_GOODS_PRICE",
    "RATE_DOWN_PAYMENT", "RATE_INTEREST_PRIMARY", "RATE_INTEREST_PRIVILEGED",
    "CNT_PAYMENT",
    "DAYS_FIRST_DRAWING", "DAYS_FIRST_DUE",
    "DAYS_LAST_DUE_1ST_VERSION", "DAYS_LAST_DUE",
    "DAYS_TERMINATION",
    "NFLAG_INSURED_ON_APPROVAL"
]
for col in num_cols:
    if col in prev_clean.columns:
        prev_clean[col] = pd.to_numeric(prev_clean[col], errors="coerce")

## Nettoyage des catégorielles

In [6]:

cat_cols = [
    "NAME_CONTRACT_TYPE", "WEEKDAY_APPR_PROCESS_START",
    "FLAG_LAST_APPL_PER_CONTRACT", "NAME_CASH_LOAN_PURPOSE",
    "NAME_CONTRACT_STATUS", "NAME_PAYMENT_TYPE",
    "CODE_REJECT_REASON", "NAME_TYPE_SUITE",
    "NAME_CLIENT_TYPE", "NAME_GOODS_CATEGORY",
    "NAME_PORTFOLIO", "NAME_PRODUCT_TYPE",
    "CHANNEL_TYPE", "NAME_SELLER_INDUSTRY",
    "NAME_YIELD_GROUP", "PRODUCT_COMBINATION"
]
for col in cat_cols:
    if col in prev_clean.columns:
        prev_clean[col] = prev_clean[col].astype("string").str.strip()

## FLAG NA

In [7]:
for col in num_cols:
    if col in prev_clean.columns:
        prev_clean[f"PREV_{col}_NA"] = prev_clean[col].isna().astype(int)

for col in cat_cols:
    if col in prev_clean.columns:
        prev_clean[f"PREV_{col}_NA"] = prev_clean[col].isna().astype(int)

## Features métiers utiles

In [8]:

# Ratio crédit accordé / demandé
prev_clean["PREV_CREDIT_TO_APPLICATION"] = (
    prev_clean["AMT_CREDIT"] / prev_clean["AMT_APPLICATION"].replace(0, np.nan)
)

# Ratio apport / prix du bien
prev_clean["PREV_DOWNPAYMENT_TO_GOODSPRICE"] = (
    prev_clean["AMT_DOWN_PAYMENT"] / prev_clean["AMT_GOODS_PRICE"].replace(0, np.nan)
)

# Ratio crédit / prix du bien
prev_clean["PREV_CREDIT_TO_GOODSPRICE"] = (
    prev_clean["AMT_CREDIT"] / prev_clean["AMT_GOODS_PRICE"].replace(0, np.nan)
)

# Durée du crédit
prev_clean["PREV_DURATION"] = prev_clean["CNT_PAYMENT"]

# Décalages
prev_clean["PREV_DECISION_TO_FIRST_DUE"] = prev_clean["DAYS_FIRST_DUE"] - prev_clean["DAYS_DECISION"]
prev_clean["PREV_DECISION_TO_LAST_DUE"]  = prev_clean["DAYS_LAST_DUE"]  - prev_clean["DAYS_DECISION"]

## Flags comportementaux

In [9]:

prev_clean["PREV_IS_REFUSED"]   = (prev_clean["NAME_CONTRACT_STATUS"] == "Refused").astype(int)
prev_clean["PREV_IS_APPROVED"]  = (prev_clean["NAME_CONTRACT_STATUS"] == "Approved").astype(int)
prev_clean["PREV_IS_CANCELED"]  = (prev_clean["NAME_CONTRACT_STATUS"] == "Canceled").astype(int)
prev_clean["PREV_IS_COMPLETED"] = (prev_clean["NAME_CONTRACT_STATUS"] == "Completed").astype(int)

# Plusieurs demandes le même jour (quand dispo)
if "NFLAG_LAST_APPL_IN_DAY" in prev_clean.columns:
    prev_clean["PREV_MULTI_APPL_DAY"] = (prev_clean["NFLAG_LAST_APPL_IN_DAY"] == 0).astype(int)
else:
    prev_clean["PREV_MULTI_APPL_DAY"] = 0


for df in (pos, inst, cc):
    if "SK_ID_PREV" in df.columns:
        df["SK_ID_PREV"] = pd.to_numeric(df["SK_ID_PREV"], errors="coerce").astype("Int64")

prev_enriched = prev_clean.merge(pos,  on="SK_ID_PREV", how="left")
prev_enriched = prev_enriched.merge(inst, on="SK_ID_PREV", how="left")
prev_enriched = prev_enriched.merge(cc,   on="SK_ID_PREV", how="left")

prev_enriched["PREV_ENRICH_HAS_NA"] = prev_enriched.isna().any(axis=1).astype(int)

print("prev_enriched:", prev_enriched.shape)

prev_enriched: (1670214, 288)


## Agrégations au niveau ID

In [10]:

num_cols_enriched = prev_enriched.select_dtypes(include=["float64", "int64", "Float64", "Int64"]).columns
num_cols_enriched = [c for c in num_cols_enriched if c not in ["SK_ID_PREV", "SK_ID_CURR"]]

agg_funcs = ["mean", "min", "max", "sum", "std"]

prev_num_agg = prev_enriched.groupby("SK_ID_CURR")[num_cols_enriched].agg(agg_funcs)
prev_num_agg.columns = [f"PREV_{col}_{stat}".upper() for col, stat in prev_num_agg.columns]

# ----------------------------
# Catégorielles (dummies)
# ----------------------------
status_cols = [
    "NAME_CONTRACT_STATUS",
    "NAME_CONTRACT_TYPE",
    "NAME_PAYMENT_TYPE",
    "NAME_CLIENT_TYPE",
    "NAME_PORTFOLIO",
    "NAME_PRODUCT_TYPE",
    "CODE_REJECT_REASON"
]
status_cols = [c for c in status_cols if c in prev_enriched.columns]

prev_cat_dummies = pd.get_dummies(
    prev_enriched[status_cols],
    prefix=[f"PREV_{c}" for c in status_cols],
    dummy_na=False
)
prev_cat_dummies["SK_ID_CURR"] = prev_enriched["SK_ID_CURR"]
prev_cat_agg = prev_cat_dummies.groupby("SK_ID_CURR").sum()

# ----------------------------
# Ratios comportementaux
# ----------------------------
prev_ratios = prev_enriched.groupby("SK_ID_CURR").agg(
    PREV_RATIO_REFUSED=("PREV_IS_REFUSED", "mean"),
    PREV_RATIO_APPROVED=("PREV_IS_APPROVED", "mean"),
    PREV_RATIO_CANCELED=("PREV_IS_CANCELED", "mean"),
    PREV_RATIO_COMPLETED=("PREV_IS_COMPLETED", "mean"),
    PREV_RATIO_MULTI_DAY=("PREV_MULTI_APPL_DAY", "mean")
)

# ----------------------------
# Count
# ----------------------------
prev_counts = prev_enriched.groupby("SK_ID_CURR").agg(
    PREV_COUNT_APPLICATIONS=("SK_ID_PREV", "count")
)

# ----------------------------
# Fusion finale
# ----------------------------
previous_final = (
    prev_num_agg
    .join(prev_cat_agg)
    .join(prev_ratios)
    .join(prev_counts)
).reset_index()

print("previous_final:", previous_final.shape)
previous_final.head()


previous_final: (338857, 1205)


,SK_ID_CURR,PREV_AMT_ANNUITY_MEAN,PREV_AMT_ANNUITY_MIN,PREV_AMT_ANNUITY_MAX,PREV_AMT_ANNUITY_SUM,PREV_AMT_ANNUITY_STD,PREV_AMT_APPLICATION_MEAN,PREV_AMT_APPLICATION_MIN,PREV_AMT_APPLICATION_MAX,PREV_AMT_APPLICATION_SUM,PREV_AMT_APPLICATION_STD,PREV_AMT_CREDIT_MEAN,PREV_AMT_CREDIT_MIN,PREV_AMT_CREDIT_MAX,PREV_AMT_CREDIT_SUM,PREV_AMT_CREDIT_STD,PREV_AMT_DOWN_PAYMENT_MEAN,PREV_AMT_DOWN_PAYMENT_MIN,PREV_AMT_DOWN_PAYMENT_MAX,PREV_AMT_DOWN_PAYMENT_SUM,PREV_AMT_DOWN_PAYMENT_STD,PREV_AMT_GOODS_PRICE_MEAN,PREV_AMT_GOODS_PRICE_MIN,PREV_AMT_GOODS_PRICE_MAX,PREV_AMT_GOODS_PRICE_SUM,PREV_AMT_GOODS_PRICE_STD,PREV_HOUR_APPR_PROCESS_START_MEAN,PREV_HOUR_APPR_PROCESS_START_MIN,PREV_HOUR_APPR_PROCESS_START_MAX,PREV_HOUR_APPR_PROCESS_START_SUM,PREV_HOUR_APPR_PROCESS_START_STD,PREV_NFLAG_LAST_APPL_IN_DAY_MEAN,PREV_NFLAG_LAST_APPL_IN_DAY_MIN,PREV_NFLAG_LAST_APPL_IN_DAY_MAX,PREV_NFLAG_LAST_APPL_IN_DAY_SUM,PREV_NFLAG_LAST_APPL_IN_DAY_STD,PREV_RATE_DOWN_PAYMENT_MEAN,PREV_RATE_DOWN_PAYMENT_MIN,PREV_RATE_DOWN_PAYMENT_MAX,PREV_RATE_DOWN_PAYMENT_SUM,PREV_RATE_DOWN_PAYMENT_STD,PREV_RATE_INTEREST_PRIMARY_MEAN,PREV_RATE_INTEREST_PRIMARY_MIN,PREV_RATE_INTEREST_PRIMARY_MAX,PREV_RATE_INTEREST_PRIMARY_SUM,PREV_RATE_INTEREST_PRIMARY_STD,PREV_RATE_INTEREST_PRIVILEGED_MEAN,PREV_RATE_INTEREST_PRIVILEGED_MIN,PREV_RATE_INTEREST_PRIVILEGED_MAX,PREV_RATE_INTEREST_PRIVILEGED_SUM,PREV_RATE_INTEREST_PRIVILEGED_STD,PREV_DAYS_DECISION_MEAN,PREV_DAYS_DECISION_MIN,PREV_DAYS_DECISION_MAX,PREV_DAYS_DECISION_SUM,PREV_DAYS_DECISION_STD,PREV_SELLERPLACE_AREA_MEAN,PREV_SELLERPLACE_AREA_MIN,PREV_SELLERPLACE_AREA_MAX,PREV_SELLERPLACE_AREA_SUM,PREV_SELLERPLACE_AREA_STD,PREV_CNT_PAYMENT_MEAN,PREV_CNT_PAYMENT_MIN,PREV_CNT_PAYMENT_MAX,PREV_CNT_PAYMENT_SUM,PREV_CNT_PAYMENT_STD,PREV_DAYS_FIRST_DRAWING_MEAN,PREV_DAYS_FIRST_DRAWING_MIN,PREV_DAYS_FIRST_DRAWING_MAX,PREV_DAYS_FIRST_DRAWING_SUM,PREV_DAYS_FIRST_DRAWING_STD,PREV_DAYS_FIRST_DUE_MEAN,PREV_DAYS_FIRST_DUE_MIN,PREV_DAYS_FIRST_DUE_MAX,PREV_DAYS_FIRST_DUE_SUM,PREV_DAYS_FIRST_DUE_STD,PREV_DAYS_LAST_DUE_1ST_VERSION_MEAN,PREV_DAYS_LAST_DUE_1ST_VERSION_MIN,PREV_DAYS_LAST_DUE_1ST_VERSION_MAX,PREV_DAYS_LAST_DUE_1ST_VERSION_SUM,PREV_DAYS_LAST_DUE_1ST_VERSION_STD,PREV_DAYS_LAST_DUE_MEAN,PREV_DAYS_LAST_DUE_MIN,PREV_DAYS_LAST_DUE_MAX,PREV_DAYS_LAST_DUE_SUM,PREV_DAYS_LAST_DUE_STD,PREV_DAYS_TERMINATION_MEAN,PREV_DAYS_TERMINATION_MIN,PREV_DAYS_TERMINATION_MAX,PREV_DAYS_TERMINATION_SUM,PREV_DAYS_TERMINATION_STD,PREV_NFLAG_INSURED_ON_APPROVAL_MEAN,PREV_NFLAG_INSURED_ON_APPROVAL_MIN,PREV_NFLAG_INSURED_ON_APPROVAL_MAX,PREV_NFLAG_INSURED_ON_APPROVAL_SUM,PREV_NFLAG_INSURED_ON_APPROVAL_STD,PREV_PREV_CREDIT_TO_APPLICATION_MEAN,PREV_PREV_CREDIT_TO_APPLICATION_MIN,PREV_PREV_CREDIT_TO_APPLICATION_MAX,PREV_PREV_CREDIT_TO_APPLICATION_SUM,PREV_PREV_CREDIT_TO_APPLICATION_STD,PREV_PREV_DOWNPAYMENT_TO_GOODSPRICE_MEAN,PREV_PREV_DOWNPAYMENT_TO_GOODSPRICE_MIN,PREV_PREV_DOWNPAYMENT_TO_GOODSPRICE_MAX,PREV_PREV_DOWNPAYMENT_TO_GOODSPRICE_SUM,PREV_PREV_DOWNPAYMENT_TO_GOODSPRICE_STD,PREV_PREV_CREDIT_TO_GOODSPRICE_MEAN,PREV_PREV_CREDIT_TO_GOODSPRICE_MIN,PREV_PREV_CREDIT_TO_GOODSPRICE_MAX,PREV_PREV_CREDIT_TO_GOODSPRICE_SUM,PREV_PREV_CREDIT_TO_GOODSPRICE_STD,PREV_PREV_DURATION_MEAN,PREV_PREV_DURATION_MIN,PREV_PREV_DURATION_MAX,PREV_PREV_DURATION_SUM,PREV_PREV_DURATION_STD,PREV_PREV_DECISION_TO_FIRST_DUE_MEAN,PREV_PREV_DECISION_TO_FIRST_DUE_MIN,PREV_PREV_DECISION_TO_FIRST_DUE_MAX,PREV_PREV_DECISION_TO_FIRST_DUE_SUM,PREV_PREV_DECISION_TO_FIRST_DUE_STD,PREV_PREV_DECISION_TO_LAST_DUE_MEAN,PREV_PREV_DECISION_TO_LAST_DUE_MIN,PREV_PREV_DECISION_TO_LAST_DUE_MAX,PREV_PREV_DECISION_TO_LAST_DUE_SUM,PREV_PREV_DECISION_TO_LAST_DUE_STD,PREV_POS_MONTHS_MIN_MEAN,PREV_POS_MONTHS_MIN_MIN,PREV_POS_MONTHS_MIN_MAX,PREV_POS_MONTHS_MIN_SUM,PREV_POS_MONTHS_MIN_STD,PREV_POS_MONTHS_MAX_MEAN,PREV_POS_MONTHS_MAX_MIN,PREV_POS_MONTHS_MAX_MAX,PREV_POS_MONTHS_MAX_SUM,PREV_POS_MONTHS_MAX_STD,PREV_POS_MONTHS_MEAN_MEAN,PREV_POS_MONTHS_MEAN_MIN,PREV_POS_MONTHS_MEAN_MAX,PREV_POS_MONTHS_MEAN_SUM,PREV_POS_MONTHS_MEAN

## Export

In [11]:

PREV_OUT_PATH = DATA_PROCESSED / "previous_final.csv"
previous_final.to_csv(PREV_OUT_PATH, index=False)
print("Export terminé :", PREV_OUT_PATH)

Export terminé : c:\Users\yoann\Documents\open classrooms\projet 6\pret a depenser v2\data\processed\previous_final.csv
